# Лабораторная работа №4
## Выполнил студент группы Поляков Вадим БВТ2002


Реализовать следующие структуры данных:
  
● <b> Стек (stack): </b>  
Операции для стека: 
<i> инициализация, проверка на пустоту, добавление нового элемента в начало, извлечение элемента из начала;  </i>  
  
● <b> Дек (двусторонняя очередь, deque):  </b>  
Операции для дека: 
<i> инициализация, проверка на пустоту, добавление нового элемента в начало, добавление нового элемента в конец, извлечение элемента из начала, извлечение элемента из конца. </i>  
  
<i> Разработать программу обработки данных, содержащихся в заранее подготовленном txt-файле, в соответствии с заданиями, применив указанную в задании структуру данных. Результат работы программы вывести на экран и сохранить в отдельном txt-файле. </i>

## Динамические структуры данных

### Stack - Значения записываются друг над другом, чтобы добраться до значения ниже - нужно убрать все значения выше. Last-In-First-Out

### Query - First-In-First-Out

### Deque - То же самое, что и Stack, только значения могут записываться с двух сторон 

In [1]:
import copy
import random
class Stack:
    def __init__(self, size):
        self.arr = ['empty'] * size
        self.count = 0
        
    def push(self, elem):
        if self.count + 1 > len(self.arr): 
            raise Exception('Элемент {}, не может быть помещен в Stack, так как он будет переполнен.'.format(elem))
        self.arr[self.count] = elem
        self.count += 1
        
    def pop(self):
        if self.count - 1 >= 0:
            elem = self.arr[self.count-1]
            self.arr[self.count-1] = 'empty'
            self.count -= 1
            if elem == None:
                return 0
            else:
                return elem
            
    def top(self):
        elem = self.arr[self.count-1]
        if elem == None:
            return 0
        else:
            return elem
                    
    def isEmpty(self):
        if self.count == 0:
                return True        
        return False

    def print(self):
        print(self.arr)
random.seed(49)        
class Deque(Stack):
    def __init__(self, size):
        self.j = -1
        Stack.__init__(self, size)
    
    def _move(self, direct):
        if direct == 'right' and self.arr[len(self.arr) - 1] == 'empty':
            nArr = ['empty'] * len(self.arr)
            j = 0
            for i in range(len(self.arr)):
                if i >= len(self.arr) - self.count:
                    nArr[i] = self.arr[j]
                    j += 1
            self.arr = nArr.copy()
        if direct == 'left' and self.arr[0] == 'empty':
            nArr = ['empty'] * len(self.arr)
            j = len(self.arr) - self.count
            for i in range(len(self.arr)):
                if j < len(self.arr):
                    nArr[i] = self.arr[j]
                    j += 1
            self.arr = nArr.copy()
            
    def push(self, elem):
        Deque._move(self, 'left')
        Stack.push(self, elem)
          
    def pop(self):
        Deque._move(self, 'left')
        return Stack.pop(self)
        
    def pushEnd(self, elem):
        if len(self.arr) < 2:
            Stack.push(self, elem)
        if self.count + 1 > len(self.arr):
            raise Exception('Элемент {}, не может быть помещен в Stack, так как он будет переполнен.'.format(elem))
        else:
            Deque._move(self, 'right')
            self.arr[len(self.arr) - self.count - 1] = elem
            self.count += 1
            
    def popEnd(self):
        if len(self.arr) - (self.count - 1) <= len(self.arr):
            Deque._move(self, 'right')
            elem = self.arr[len(self.arr) - self.count] 
            self.arr[len(self.arr) - self.count] = 'empty'
            self.count -= 1
            return elem

### Задание №1
Отсортировать строки файла, содержащие названия книг, в алфавитном порядке с использованием двух деков.

In [2]:
books = []

f = open('input_task1.txt', encoding="utf-8")
for line in f:
    books.append(line[:-1])
f.close()

d1 = Deque(len(books))
d2 = Deque(len(books))

for book in range(len(books)):
    d1.push(books[book])

def sort(fr, to): 
    # Перекидываем первый элемент из одной деки в другую
    book = fr.pop()
    lastBook = book
    to.push(book)
    
    for i in range(len(books)-1):
        for j in range(min(len(lastBook),len(book))):
            book = fr.pop()
            if ord(lastBook[j]) != ord(book[j]):
                if ord(lastBook[j]) < ord(book[j]):
                    to.pushEnd(book)
                if ord(lastBook[j]) > ord(book[j]):
                    to.push(book)
                lastBook = book
                break
            else:
                fr.push(book)
                
    # Отсортированы ли элементы уже?
    isSorted = True
    book = to.pop()
    lastBook = book
    fr.push(book)
    
    for i in range(len(books)-1):
        for j in range(min(len(lastBook),len(book))):
            book = to.pop()
            if ord(lastBook[j]) != ord(book[j]):
                if ord(lastBook[j]) > ord(book[j]):
                    isSorted = False
                fr.pushEnd(book)
                lastBook = book
                break
            else:
                to.push(book)
    
    #Сортируем, пока не отсортируется
    if (not isSorted): 
        return sort(fr,to)
    
# Сортируем список книг
sort(d1,d2)

# Открываем файл для записи 
f = open('output_task1.txt', 'w', encoding="utf-8")
# Заносим отсортированные книги в файл
for i in range(d1.count):
    print(d1.pop(), file=f)
f.close()

### Задание №2
Дек содержит последовательность символов для шифровки сообщений. Дан текстовый файл, содержащий зашифрованное сообщение. Пользуясь деком, расшифровать текст. 
Известно, что при шифровке каждый символ сообщения заменялся следующим за ним в деке по часовой стрелке через один.

In [3]:
alphabet = list("ЙФЯЦЫЧУВСКАМЕПИНРТГОЬШЛБЩДЮЗЖХЭЪйцукенгшщзхъэждлрпавыфячсмитьбю.?!春節源於上古蠟（通臘）祭距今已有四五千年歷史用以祭農神兼及盡春來人們就要殺掉牲畜，祭祀祖宗和上天祈求新一年可以風調雨順無災無禍收成興旺")
random.shuffle(alphabet)
alphabet = ''.join(alphabet)
print("Шифровальный массив:", alphabet)

code_array = Deque(len(alphabet))
for i in range(len(alphabet)):
    code_array.push(alphabet[i])

def encode(t,d):
    t = list(t)
    for i in range(len(t)):
        for j in range(d.count):
            c = d.pop()
            if c == t[i]:
                d.push(c)
                d.push(d.popEnd())
                c = d.popEnd()
                t[i] = c
            d.pushEnd(c)
    return ''.join(t)

message = "Сколько еще раз нужно разбиться вдребезги, чтобы наконец-то заслужить майонез с кетчупом Нежный?"

with open("input_task2.txt", "w", encoding="utf-8") as file:
    print(encode(message,code_array), file = file)

Шифровальный массив: 春ЛЩ上祭一以Н調求和И順節К雨成аъМ禍гЙГ上宗ПшВц歷天бХ，年н農Ш通РАд以п掉ЭмЬЗсхьБ古э祭Ы!ф殺年тДФ風Я春就.臘ч有們祈ю牲щ源）災й蠟С盡Ю要新興ы千У今距祀五уж人зр可я無旺神ЪЦвл（е祭祖收?於已無兼иОк及來Ч史Е四Ж畜用Т


In [4]:
with open("input_task2.txt", "r", encoding="utf-8") as file:
    s = file.readline()
    
print("Зашифрованное сообщение:", s)

def decode(t,d):
    t = list(t)
    for i in range(len(t)):
        for j in range(d.count):
            c = d.pop()
            if c == t[i]:
                d.pushEnd(c)
                d.pushEnd(d.pop())
                c = d.pop()
                t[i] = c
            d.pushEnd(c)
    return ''.join(t)

print("Исходное сообщение:", decode(s,code_array))

Зашифрованное сообщение: Ю來ое古來о 祖）祖 яМ可 Ш人зШо яМ可，кФ古ь旺 （пя祖，祖可Гк, 們Фо，У ШМ來оШ祖天-Фо 可Мье人зкФ古 ЗМСоШ祖可 ь 來祖Ф們人ЭоЗ 求祖зШУС已

Исходное сообщение: Сколько еще раз нужно разбиться вдребезги, чтобы наконец-то заслужить майонез с кетчупом Нежный?



### Задание №3
Даны три стержня и n дисков различного размера. Диски можно надевать на стержни, образуя из них башни. Перенести n дисков со стержня А на стержень С, сохранив их первоначальный порядок. При переносе дисков необходимо соблюдать следующие правила:
- на каждом шаге со стержня на стержень переносить только один диск;
- диск нельзя помещать на диск меньшего размера;
- для промежуточного хранения можно использовать стержень В.
Реализовать алгоритм, используя три стека вместо стержней А, В, С. Информация о дисках хранится в исходном файле.

In [5]:
print("Ханойская башня")

disks = 4
A = Stack(disks) 
B = Stack(disks) 
C = Stack(disks) 

with open("input_task3.txt", "r", encoding="utf-8") as file:
    s = file.readline()
    
s = s.split(', ')

for i in range(len(s)):
    A.push(s[i])


def move(st1, st2, let1, let2):
    if st1.count == 0 and st2.count > 0:
        st1.push(st2.pop())
    elif st1.count > 0 and st2.count == 0:
        st2.push(st1.pop())
    elif st1.top() > st2.top():
        st1.push(st2.pop())
    else:
        st2.push(st1.pop())
    
    print(let1, ":", st1.arr)
    print(let2, ":", st2.arr)
    print("")
    
if disks % 2 == 0:
    while C.count != disks:
        move(A, B, 'A', 'B')
        move(A, C, 'A', 'C')
        move(B, C, 'B', 'C')
else:
    while C.count != disks:
        move(A, C, 'A', 'C')
        if C.count == disks:
            break
        move(A, B, 'A', 'B')
        move(B, C, 'B', 'C')

Ханойская башня
A : ['4', '3', '2', 'empty']
B : ['1', 'empty', 'empty', 'empty']

A : ['4', '3', 'empty', 'empty']
C : ['2', 'empty', 'empty', 'empty']

B : ['empty', 'empty', 'empty', 'empty']
C : ['2', '1', 'empty', 'empty']

A : ['4', 'empty', 'empty', 'empty']
B : ['3', 'empty', 'empty', 'empty']

A : ['4', '1', 'empty', 'empty']
C : ['2', 'empty', 'empty', 'empty']

B : ['3', '2', 'empty', 'empty']
C : ['empty', 'empty', 'empty', 'empty']

A : ['4', 'empty', 'empty', 'empty']
B : ['3', '2', '1', 'empty']

A : ['empty', 'empty', 'empty', 'empty']
C : ['4', 'empty', 'empty', 'empty']

B : ['3', '2', 'empty', 'empty']
C : ['4', '1', 'empty', 'empty']

A : ['2', 'empty', 'empty', 'empty']
B : ['3', 'empty', 'empty', 'empty']

A : ['2', '1', 'empty', 'empty']
C : ['4', 'empty', 'empty', 'empty']

B : ['empty', 'empty', 'empty', 'empty']
C : ['4', '3', 'empty', 'empty']

A : ['2', 'empty', 'empty', 'empty']
B : ['1', 'empty', 'empty', 'empty']

A : ['empty', 'empty', 'empty', 'empty']


### Задание №4
Дан текстовый файл с программой на алгоритмическом языке. За один просмотр файла проверить баланс круглых скобок в тексте, используя стек.

In [ ]:
pre_line = []
f = open('input_task4.txt', encoding="utf-8")

for line in f:
    pre_line.append(line[:-1])
f.close()

line = "".join([j for sub in pre_line for j in sub])

def isBracketsBalance(s):
    st = Stack(len(s))
    
    for c in s:
        if c == '(':
            st.push(c)
        if c == ')':
            if st.isEmpty():
                return False
            st.pop()
            
    return st.isEmpty()

print(isBracketsBalance(line))

### Задание №5
Дан текстовый файл с программой на алгоритмическом языке. За один просмотр файла проверить баланс квадратных скобок в тексте, используя дек.

In [ ]:
pre_line = []
f = open('input_task5.txt', encoding="utf-8")

for line in f:
    pre_line.append(line[:-1])
f.close()

line = "".join([j for sub in pre_line for j in sub])

def isBracketsBalance(s):
    st = Stack(len(s))
    
    for c in s:
        if c == '[':
            st.push(c)
            
        if c == ']':
            if st.isEmpty():
                return False
            st.pop()
            
    return st.isEmpty()

print(isBracketsBalance(line))

### Задание №6
Дан файл из символов. Используя стек, за один просмотр файла напечатать сначала все цифры, затем все буквы, и, наконец, все остальные символы, сохраняя исходный порядок в каждой группе символов.

In [6]:
full_line = ''
symbs = ''
nums = ''
letters = ''

f = open('input_task6.txt', encoding="utf-8")
for line in f:
    full_line += line
f.close()

stacked_line = Stack(len(full_line))
for c in reversed(full_line):
    stacked_line.push(c)

for i in range(len(full_line)):
    # Если символ это цифра
    if ord(stacked_line.top()) >= 48 and ord(stacked_line.top()) < 58:
        nums += stacked_line.pop()
    # Если это спец. символ. В том числе и пробелы
    elif ord(stacked_line.top()) >= 32 and ord(stacked_line.top()) < 48 or ord(stacked_line.top()) >= 58 and ord(stacked_line.top()) < 65:
        symbs += stacked_line.pop()
        
    else:
        letters += stacked_line.pop()

print("Все цифры:")
print("")
print(nums)
print("")
print("Все буквы:")
print("")
print(letters)
print("")
print("Все спец. символы:")
print("")
print(symbs)

Все цифры:

86545087875

Все буквы:

ТекстПесни
СугаАвгустДэ

Theycallmenewthang
신병왔다짐을받어
wholeworldconcert
꽤먹히는AsianaAsia
youcouldbemynewthang

근무태만형들과는달러
유명인의하극상damn
쎈놈만덤벼
어떤이는내가이자릴쉽게앉았다고해
Fckyou난성공과
거리가먼형들사이눈엣가시네
솔직히싸이하누월쪽팔려
이제일년에만장팔어
Kpop이란카테고리
날담기에는사이즈가달러whoo
그래앞서
가고싶다면firstclass예약해봐
Myseatisbusiness
넌이코노미평생내뒤지kissingmy
다음목표는빌보드
Braziltonewyork
꽤쉴틈없는내passport

Все спец. символы:

            ,                       *            -                     ***          


### Задание №7
Дан файл из целых чисел. Используя дек, за один просмотр файла напечатать сначала все отрицательные числа, затем все положительные числа, сохраняя исходный порядок в каждой группе.

In [7]:
t = ''
pos = ''
neg = ''

# Записываем весь текст из файла в переменную
f = open('input_task7.txt', encoding="utf-8")
for line in f:
    t += line
f.close()

st = Stack(len(t))
for c in reversed(t):
    st.push(c)

isNegAdd = False

for i in range(len(t)):
    # Если это пробел
    if ord(st.top()) == 32:
        isNegAdd = False
        if len(pos) > 0 and ord(pos[len(pos)-1]) != 32:
            pos += ' '
        if len(neg) > 0 and ord(neg[len(neg)-1]) != 32:
            neg += ' '
        st.pop()
    # Если это минус
    elif ord(st.top()) == 45:
        isNegAdd = True
        neg += st.pop()
    else:
        if isNegAdd:
            neg += st.pop()
        else:
            pos += st.pop()
            
print("Отрицательные числа:", neg)
print("Положительные числа:", pos)

Отрицательные числа: -5 -51 -5813 -931 -835 -411 -173 -538888 -51 -1 -58
Положительные числа: 1 9 47 1 5 681 503 3 585 1204 3 580 3915 1 9583 23 


### Задание №8
Дан текстовый файл. Используя стек, сформировать новый текстовый файл, содержащий строки исходного файла, записанные в обратном порядке: первая строка становится последней, вторая – предпоследней и т.д.

In [8]:
rows = []
    
input_file = open('input_task8.txt', encoding="utf-8")

for row in input_file:
    print(row)
    rows.append(row[:-3])

s = Stack(len(rows))
for i in range(len(rows)):
    s.push(rows[i])

output_file = open('output_task8.txt', 'w', encoding="utf-8")

for i in range(s.count):
    print(s.pop(), file=output_file)
    
output_file.close()

한입으로 두말 여자들은 널 보고 불러 언니

hanibeuro dumar yeojadeureun neor bogo bulleo eonni

Two different things with one mouth, girls call you ‘unnie’

“Two different things with one mouth” is something that has been traditionally tabooed for men in Korea. 

There are even old sayings such as 남아일언중천금 (男兒一言重千金;

a real guy’s one word carries the weight of a thousand pieces of gold). 

It is obviously an outcome of gender-biased prejudice, but is still widely used. 

Some would jokingly call a man who changes his word “a woman.” 

With that said, “girls call you unnie” because “you” said two different things from one mouth. 

“Since you spoke two different things with one mouth, you’re not a real man. 

So, I’m going to treat you as a woman, and those girls will call you unnie.”


### Вывод
